# Function Calling


In [ ]:
# !pip install pandas "mistralai>=0.1.2"

### Load API key

In [ ]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [15]:
import pandas as pd

In [16]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [17]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


#### How you might answer data questions without function calling
- Not recommended, but an example to serve as a contrast to function calling.

In [4]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [ ]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [ ]:
response = mistral(prompt)
print(response)

## Step 1. User: specify tools and query

### Tools

- You can define all tools that you might want the model to call.

In [18]:
import json

In [23]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [34]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [35]:
type(status)

str

In [36]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [37]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


- You can outline the function specifications with a JSON schema.

In [38]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [39]:
type(tool_payment_status)

dict

In [40]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [41]:
type(tool_payment_status)

dict

In [42]:
tools = [tool_payment_status, tool_payment_date]

In [43]:
type(tools)

list

In [44]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

### functools

In [80]:
import functools

In [81]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [82]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [83]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

### User query

- Example: “What’s the status of my transaction?”

In [ ]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

## Step 2. Model: Generate function arguments 

In [ ]:
from mistralai.client import MistralClient

model = "mistral-large-latest"

client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"), endpoint=os.getenv("DLAI_MISTRAL_API_ENDPOINT"))

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

In [ ]:
response.choices[0].message.content

### Save the chat history

In [ ]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

In [ ]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [ ]:
response

In [ ]:
response.choices[0].message

In [ ]:
chat_history.append(response.choices[0].message)

- Notice these fields:
- `name='retrieve_payment_status'`
- `arguments='{"transaction_id": "T1001"}'`

## Step 3. User: Execute function to obtain tool results

- Currently, the user is the one who will execute these functions (the model will not execute these functions on its own).

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

In [ ]:
tool_function.name

In [ ]:
tool_function.arguments

- The function arguments are expected to be in a Python dictionary and not a string.
- To make this string into a dictionary, you can use `json.loads()`  

In [ ]:
args = json.loads(tool_function.arguments)
print(args)

- Recall the functools dictionary that you made earlier

```Python
import functools
names_to_functions = {
    "retrieve_payment_status": 
      functools.partial(retrieve_payment_status, df=df),
    
    "retrieve_payment_date": 
      functools.partial(retrieve_payment_date, df=df),
}
```

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result

- The output of the function call can be saved as a chat message, with the role "tool".

In [ ]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [ ]:
chat_history

## Step 4. Model: Generate final answer
- The model can now reply to the user query, given the information provided by the "tool".

In [ ]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

### Try it for yourself!
- Try asking another question about the data, such as "how much did I pay my recent order?
  - You can be customer "T1002", for instance

### Using Local Models

### Create a grammar file called json.gbnf
The json.gbnf in llama.cpp is a grammar format used to ensure that model outputs adhere to specific structures, in this case valid JSON. Using grammar is crucial for applications requiring structured and accurate LLM model outputs.

Place the json.gbn ffile in the same folder as your python file and downloaded model.

## The MethodCaller
The MethodCaller.run method accepts a string that contains natural language and function to be executed. The query should contain the parameters that are passed to the function.

It's important to ensure that the function is annotated using standard Python docstring format. This annotation communicate in providing clear documentation and understanding of the function's purpose, its parameters, and the expected return type to the LLM.

In [102]:
import inspect
from typing import Any, Callable, Dict
import json
import llama_cpp
from llama_cpp import LlamaGrammar

class MethodCaller:
    def __init__(self, model: Any, context: Dict[str, Any] = None):
        self.grammar = LlamaGrammar.from_file("json.gbnf")
        self.model = model
        self.context = context or {}

    def get_schema(self, item: Callable) -> Dict[str, Any]:
        signature = inspect.signature(item)
        parameters = signature.parameters
        
        schema = {
            "name": item.__name__,
            "description": str(inspect.getdoc(item)),
            "parameters": {},
            "output": str(signature.return_annotation),
        }
        
        for name, param in parameters.items():
            if name in self.context:
                continue  # Skip parameters that are in the context
            schema["parameters"][name] = {
                "type": str(param.annotation),
                "default": str(param.default) if param.default is not param.empty else None
            }
        
        return schema

    def run(self, query: str, function: Callable):
        schema = self.get_schema(function)
         
        prompt = f"""
            You are a helpful assistant designed to output JSON.
            Given the following function schema
            << {schema} >>
            and query
            << {query} >>
            extract the parameters values from the query, in a valid JSON format.
            Only include parameters that are in the schema, do not include any extra parameters.
            Example:
            Input:
            query: "How is the weather in Hawaii right now in International units?"
            schema:
            {{
                "name": "get_weather",
                "description": "Useful to get the weather in a specific location",
                "parameters": {{
                    "location": {{"type": "<class 'str'>", "default": None}},
                    "degree": {{"type": "<class 'str'>", "default": "Celsius"}}
                }},
                "output": "<class 'str'>",
            }}
            Result: {{
                "location": "Hawaii",
                "degree": "Celsius"
            }}
            Input:
            query: {query}
            schema: {schema}
            Result:
        """
        completion = self.model.create_chat_completion(
            messages=[{
                "role": "user",
                "content": prompt
            }],
            temperature=0.1,
            max_tokens=500,
            grammar=self.grammar,
            stream=False,
        )
        output = completion["choices"][0]["message"]["content"]
        output = output.replace("'", '"').strip().rstrip(",")
      
        function_inputs = json.loads(output)
        print(function_inputs)
        
        # Combine extracted parameters with context
        all_inputs = {**self.context, **function_inputs}
        
        return function(**all_inputs)

In [103]:
# Function to be called described using the Python docstring format   
def add_two_numbers( first_number: int, second_number: int ) -> str:
    """
    Adds two numbers together.

    :param first_number: The first number to add.
    :type first_number: int

    :param second_number: The second number to add.
    :type second_number: int

    :return: The sum of the two numbers.
    """
    return first_number + second_number 

In [97]:
model = llama_cpp.Llama(
    model_path="../Models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=32000,
    verbose=False,
)

In [104]:
fun = MethodCaller( model )

result = fun.run("Add 2 and 5", add_two_numbers)
print(result)

result = fun.run("Add 66 and seventy", add_two_numbers)
print(result)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{'first_number': 2, 'second_number': 5}
7
{'first_number': 66, 'second_number': 70}
136


In [92]:
data = df

In [76]:
fun = MethodCaller( model )

result = fun.run("Using {data} What is the status of my payment", retrieve_payment_status)
print(result)



from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{
"transaction_id": "my payment"
}


In [105]:
model = llama_cpp.Llama(
    model_path="../Models/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=8000,
    verbose=False,
)

In [106]:
fun = MethodCaller( model )

result = fun.run("Add 2 and 5", add_two_numbers)
print(result)

result = fun.run("Add 66 and seventy", add_two_numbers)
print(result)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{'first_number': 2, 'second_number': 5}
7
{'first_number': 66, 'second_number': 70}
136


In [107]:
model = llama_cpp.Llama(
    model_path="../Models/Hermes-2-Pro/Hermes-2-Pro-Llama-3-8B-GGUF/Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=8000,
    verbose=False,
)

In [108]:
fun = MethodCaller( model )

result = fun.run("Add 2 and 5", add_two_numbers)
print(result)

result = fun.run("Add 66 and seventy", add_two_numbers)
print(result)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{'first_number': 2, 'second_number': 5}
7
{'first_number': 66, 'second_number': 70}
136


In [115]:
# Create the DataFrame
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

# Initialize the MethodCaller with the model and context
fun = MethodCaller(model, context={"df": df})

# Define the retrieve_payment_status function
def retrieve_payment_status(df: pd.DataFrame, transaction_id: str) -> str:
    """
    Retrieves the payment status for a given transaction ID.
    :param df: The DataFrame containing the payment data.
    :param transaction_id: The ID of the transaction to look up.
    :return: A JSON string containing the payment status or an error message.
    """
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

# Run the function
# result = fun.run("What is the payment status for transaction T1002?", retrieve_payment_status)
# print(result)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

In [110]:
result = fun.run(" what is the payment status of transaction_id 'T1002'?", retrieve_payment_status)
print(result)



{'transaction_id': 'T1002'}
{"status": "Unpaid"}


In [111]:
model = llama_cpp.Llama(
    model_path="../Models/gorilla-llm/gorilla-openfunctions-v2-gguf/gorilla-openfunctions-v2-q4_K_S.gguf",
    n_gpu_layers=-1,
    n_ctx=8000,
    verbose=False,
)

In [112]:
fun = MethodCaller( model )

result = fun.run("Add 2 and 5", add_two_numbers)
print(result)

result = fun.run("Add 66 and seventy", add_two_numbers)
print(result)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

{'first_number': 2, 'second_number': 5}
7
{'first_number': 66, 'second_number': 70}
136


In [117]:
result = fun.run(" what is the payment status of transaction_id 'T1003'?", retrieve_payment_status)
print(result)


{'transaction_id': 'T1003'}
{"status": "Paid"}
